In [14]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=(960, 540)):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize((960, 540), Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = []

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2 + 1], images[i*2], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("sora_inpaint", f"幻灯片{path}.png") for path in range(1, 13)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'inpainting.gif', duration=500, steps=10)


In [13]:
from PIL import Image, ImageFilter
import numpy as np
import os

# def linear_blend(img1, img2, alpha):
#     """Blend two images with a given alpha."""
#     return Image.fromarray((alpha * np.array(img1) + (1 - alpha) * np.array(img2)).astype('uint8'))

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=(960, 540)):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize((960, 540), Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = [images[0]]

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2+1 + 1], images[i*2+1], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("outpainting-1", f"幻灯片{path}.png") for path in range(1, 12)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'outpainting-1.gif', duration=500, steps=10)


In [12]:
from PIL import Image, ImageFilter
import numpy as np
import os

# def linear_blend(img1, img2, alpha):
#     """Blend two images with a given alpha."""
#     return Image.fromarray((alpha * np.array(img1) + (1 - alpha) * np.array(img2)).astype('uint8'))

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=(960, 540)):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize((960, 540), Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = [images[0]]

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2+1 + 1], images[i*2+1], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("outpainting-2", f"幻灯片{path}.png") for path in range(1, 12)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'outpainting-2.gif', duration=500, steps=10)


In [11]:
from PIL import Image, ImageFilter
import numpy as np
import os

size = (960, 540)

# def linear_blend(img1, img2, alpha):
#     """Blend two images with a given alpha."""
#     return Image.fromarray((alpha * np.array(img1) + (1 - alpha) * np.array(img2)).astype('uint8'))

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=size):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize(size, Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = [images[0]]

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2+1 + 1], images[i*2+1], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("outpainting-3", f"幻灯片{path}.png") for path in range(1, 16)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'outpainting-3.gif', duration=500, steps=10)


In [15]:
from PIL import Image, ImageFilter
import numpy as np
import os

size = (960, 540)

# def linear_blend(img1, img2, alpha):
#     """Blend two images with a given alpha."""
#     return Image.fromarray((alpha * np.array(img1) + (1 - alpha) * np.array(img2)).astype('uint8'))

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=size):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize(size, Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = [images[0]]

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2+1 + 1], images[i*2+1], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("outpainting-4", f"幻灯片{path}.png") for path in range(1, 16)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'outpainting-4.gif', duration=500, steps=10)


In [16]:
from PIL import Image, ImageFilter
import numpy as np
import os

size = (960, 540)

# def linear_blend(img1, img2, alpha):
#     """Blend two images with a given alpha."""
#     return Image.fromarray((alpha * np.array(img1) + (1 - alpha) * np.array(img2)).astype('uint8'))

def apply_dithering(image):
    """Apply dithering to the image to reduce banding."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    noise = np.random.normal(0, 5, (image.height, image.width, 3))  # 调整噪声参数
    noisy_image = np.array(image) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
    return Image.fromarray(noisy_image)


def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction to the image."""
    inv_gamma = 1.0 / gamma
    lut = [int((i / 255.0) ** inv_gamma * 255) for i in range(256)]

    if image.mode in ['RGB', 'RGBA']:
        lut = lut * 3  # 对于 RGB 或 RGBA 图像，我们需要 3 个通道的 LUT
        if image.mode == 'RGBA':
            lut += [i for i in range(256)]  # 对于 RGBA，透明度通道保持不变

    return image.point(lut)


def linear_blend(img1, img2, alpha, gamma=2.2):
    """Blend two images with a given alpha and apply gamma correction."""
    img1_corrected = adjust_gamma(img1, gamma)
    img2_corrected = adjust_gamma(img2, gamma)
    blended = Image.fromarray((alpha * np.array(img1_corrected) + (1 - alpha) * np.array(img2_corrected)).astype('uint8'))
    blended_corrected = adjust_gamma(blended, 1/gamma)
    return apply_dithering(blended_corrected)


def thumbnail_and_pad(image_path, target_size=size):
    # 打开原始图像
    image = Image.open(image_path)

    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # 缩放图像
    image.thumbnail(target_size, )

    # 创建一个透明背景图像
    background = Image.new('RGBA', target_size, (0, 0, 0, 0))

    # 计算居中位置
    x = (target_size[0] - image.width) // 2
    y = (target_size[1] - image.height) // 2

    # 将缩放后的图像粘贴到背景上
    background.paste(image, (x, y), image)

    return background


def create_gradient_gif(image_filenames, gif_filename, duration, steps):
    """
    Create a gradient GIF from a list of images.

    :param image_filenames: List of image filenames.
    :param gif_filename: Filename for the output GIF.
    :param duration: Duration of each transition in the GIF.
    :param steps: Number of steps in each transition.
    """
    # images = [thumbnail_and_pad(filename) for filename in image_filenames if "DS_Store" not in filename]
    images = [Image.open(filename).convert("RGB").resize(size, Image.Resampling.BICUBIC) for filename in image_filenames if "DS_Store" not in filename]
    frames = [images[0]]

    for i in range(len(images) // 2):
        for step in range(steps):
            alpha = step / float(steps - 1)
            new_frame = linear_blend(images[i*2+1 + 1], images[i*2+1], alpha)
            frames.append(new_frame)

    frames[0].save(gif_filename, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0, disposal=2)

# Example usage
image_filenames = [os.path.join("outpainting-5", f"幻灯片{path}.png") for path in range(1, 16)]  # Replace with your image paths
create_gradient_gif(image_filenames, 'outpainting-5.gif', duration=500, steps=10)
